In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_free_goods_promotion 
--PARTITION BY valid_from CLUSTER BY article, add_article, vendor_id AS
-- SITE LEVEL 

SELECT 
    a.lifnr vendor_id,
    a.matnr article,
    b.knrmm min_qty,
    b.knrme free_goods_unit,
    b.knrnm free_goods_qty,
    b.knrzm add_qty,
    b.knrez add_unit,
    b.knrmat add_article,
    CAST(NULL AS STRING) supply_region,
    a.werks store_id,
    a.datab valid_from,
    a.datbi valid_to,
    b.knr_ak promo_id,
    prt.description promo_desc,
    prt.theme promo_theme,
    0 pri
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm37 a 
LEFT JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm35 b ON b.knumh = a.knumh
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(txtmd) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) txtmd FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_trt_promo) pr ON pr.rt_promo = b.knr_ak
LEFT JOIN (
    SELECT DISTINCT 
        promotion, 
        FIRST_VALUE(theme) OVER (PARTITION BY promotion ORDER BY created_at DESC) theme,
        FIRST_VALUE(description) OVER (PARTITION BY promotion ORDER BY created_at DESC) description 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm472) prt ON prt.promotion = b.knr_ak
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(rt_promoct) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) rt_promoct FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo) typ ON typ.rt_promo = b.knr_ak
UNION ALL
-- REGION LEVEL
SELECT 
    a.lifnr vendor_id,
    a.matnr article,
    b.knrmm min_qty,
    b.knrme free_goods_unit,
    b.knrnm free_goods_qty,
    b.knrzm add_qty,
    b.knrez add_unit,
    b.knrmat add_article,
    a.zz_supply_region supply_region,
    CAST(NULL AS STRING) site,
    a.datab valid_from,
    a.datbi valid_to,
    b.knr_ak promo_id,
    prt.description promo_desc,
    prt.theme promo_theme,
    1 pri
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm38 a 
LEFT JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm35 b ON b.knumh = a.knumh
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(txtmd) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) txtmd FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_trt_promo) pr ON pr.rt_promo = b.knr_ak
LEFT JOIN (
    SELECT DISTINCT 
        promotion, 
        FIRST_VALUE(theme) OVER (PARTITION BY promotion ORDER BY created_at DESC) theme,
        FIRST_VALUE(description) OVER (PARTITION BY promotion ORDER BY created_at DESC) description 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm472) prt ON prt.promotion = b.knr_ak
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(rt_promoct) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) rt_promoct FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo) typ ON typ.rt_promo = b.knr_ak
UNION ALL
-- FULL LEVEL
SELECT 
    a.lifnr vendor_id,
    a.matnr article,
    b.knrmm min_qty,
    b.knrme free_goods_unit,
    b.knrnm free_goods_qty,
    b.knrzm add_qty,
    b.knrez add_unit,
    b.knrmat add_article,
    CAST(NULL AS STRING) supply_region,
    CAST(NULL AS STRING) site,
    a.datab valid_from,
    a.datbi valid_to,
    b.knr_ak promo_id,
    prt.description promo_desc,
    prt.theme promo_theme,
    2 pri
FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm36 a 
LEFT JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm35 b ON b.knumh = a.knumh
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(txtmd) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) txtmd FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_trt_promo) pr ON pr.rt_promo = b.knr_ak
LEFT JOIN (
    SELECT DISTINCT 
        promotion, 
        FIRST_VALUE(theme) OVER (PARTITION BY promotion ORDER BY created_at DESC) theme,
        FIRST_VALUE(description) OVER (PARTITION BY promotion ORDER BY created_at DESC) description 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_avccmdm472) prt ON prt.promotion = b.knr_ak
LEFT JOIN (SELECT DISTINCT rt_promo, FIRST_VALUE(rt_promoct) OVER (PARTITION BY rt_promo ORDER BY end_date DESC) rt_promoct FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_prt_promo) typ ON typ.rt_promo = b.knr_ak ''')